In [1]:
from google.colab import drive
drive.mount('/root/gdrive')

Drive already mounted at /root/gdrive; to attempt to forcibly remount, call drive.mount("/root/gdrive", force_remount=True).


In [2]:
%cd ~/gdrive/MyDrive/'_ECE 570'/Project/ops/voxel-tf-ops/
%ls

/root/gdrive/MyDrive/_ECE 570/Project/ops/voxel-tf-ops
avg_vox/          cuda_includes/  MANIFEST.in     tf/               WORKSPACE
BUILD             gpu/            README.md       third_party/
build_pip_pkg.sh  LICENSE         setup.py        trilinear_devox/
configure.sh      Makefile        terminal.ipynb  utils/


In [3]:
!mkdir -p /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include

In [3]:
!cp -r cuda_includes/*.h /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include
!ls /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include

cuda_fp16.h  cuda.h


In [ ]:
# TF 2.7 had 3 errors in tensor_types.h, downgraded to TF 2.6 and works!
!pip uninstall tensorflow
!pip install tensorflow==2.6.2

In [3]:
!python3 -c 'import tensorflow as tf; print(tf.__version__)' 

2.6.2


In [3]:
import tensorflow as tf

# tf.debugging.set_log_device_placement(True)
gpu_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


# Voxelization

In [ ]:
!make clean_vox

In [5]:
from tensorflow.python.framework.errors_impl import NotFoundError

avg_vox_ops_so_path = './avg_vox/python/ops/_avg_vox_ops.so'
try:
  avg_vox_ops = tf.load_op_library(avg_vox_ops_so_path)
except NotFoundError as e:
  print('Successfully cleaned!')
else:
  print('Colab is still seeing old .so')

Colab is still seeing old .so


In [ ]:
!make vox_gpu_only

In [ ]:
!make vox_op

In [9]:
!make vox_test

python avg_vox/python/ops/avg_vox_ops_test.py
2021-11-15 02:49:09.828454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 02:49:09.836988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 02:49:09.837842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] AvgVoxTest.test_avg_voxelize_forward
2021-11-15 02:49:09.841346: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU i

In [8]:
# from avg_vox.python.ops.avg_vox_ops import avg_voxelize_forward

avg_vox_ops = tf.load_op_library(avg_vox_ops_so_path)
# avg_vox_ops = tf.load_op_library('/root/gdrive/My Drive/_ECE 570/Project/ops/voxel-tf-ops/avg_vox/python/ops/_avg_vox_ops.so')
avg_voxelize_forward = avg_vox_ops.avg_vox_forward
avg_voxelize_backward = avg_vox_ops.avg_vox_backward

 Note: We only see 1, and 2, 3 in `out` because of the "clipping" that was
       added in GridStatsKernel. More specifically, the equation for
       calculating a point's voxel index may give a voxel index greater
       than R**3 (point (4,4,4) in this case). If that occurs, we do not
       increment the element in `cnt` corresponding to that voxel. Thus, we
       do not see the corresponding feature for that point in `out`.

In [7]:
###################
# Setup Constants #
###################
B = 2
C = 5
N = 4
R = 5

# [B, C, N] = [2, 5, 4]
features = tf.constant([[[1, 2, 3, 4]]], dtype=tf.float32)
features = tf.repeat(tf.repeat(features, 5, axis=1), 2, axis=0)

# [B, 3, N] = [2, 3, 4]
coords = tf.constant([[[1, 2, 3, 4]]], dtype=tf.int32)
coords = tf.repeat(tf.repeat(coords, 3, axis=1), 2, axis=0)

resolution = tf.constant(R)

##################
# Call Operation #
##################
out, ind, cnt = avg_voxelize_forward(features, coords, resolution)

#################################################
# Get Gradients of Arbitrary Loss WRT Op Output #
#################################################
mse = tf.keras.losses.MeanSquaredError()
ones_label = tf.ones(shape=[B, C, R**3])
with tf.GradientTape() as tape:
  tape.watch(out)
  loss = mse(ones_label, out)
  dL_dout = tape.gradient(loss, out)

####################################################
# Get Gradients of Loss WRT `features` input of Op #
####################################################
features_grad = avg_voxelize_backward(dL_dout, ind, cnt)

#####################
# Print Out Tensors #
#####################
print("\n\n--------- Forward ---------\n")

print("\nind\n---")
print(ind)
print()

print("cnt\n---")
print(cnt)
print()

print("out\n---")
print(out)
print()

print("--------- Backward ---------\n")

print("dL/dout\n------")
print(dL_dout)
print()

print("features graient\n----------------")
print(features_grad)
print("\n")




--------- Forward ---------


ind
---
tf.Tensor(
[[ 31  62  93 124]
 [ 31  62  93 124]], shape=(2, 4), dtype=int32)

cnt
---
tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]], shape=(2, 125), dtype=int32)

out
---
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]]

 [[0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]
  [0. 0. 0. ... 0. 0. 4.]]], shape=(2, 5, 125), dtype=flo

# Devoxelization

In [6]:
!make clean_devox

rm -f trilinear_devox/python/ops/_trilinear_devox_ops.cu.o trilinear_devox/python/ops/_trilinear_devox_ops.so 


In [4]:
from tensorflow.python.framework.errors_impl import NotFoundError

trilinear_devox_ops_so_path = './trilinear_devox/python/ops/_trilinear_devox_ops.so'
try:
  trilinear_devox_ops = tf.load_op_library(trilinear_devox_ops_so_path)
except NotFoundError as e:
  print('Successfully cleaned!')
else:
  print('Colab is still seeing old .so')

Successfully cleaned!


In [ ]:
!make devox_gpu_only

In [5]:
!make devox_op

nvcc -std=c++11 -c -o trilinear_devox/python/ops/_trilinear_devox_ops.cu.o trilinear_devox/cc/trilinear_devox_kernels.cu.cc  -I/usr/local/lib/python3.7/dist-packages/tensorflow/include -D_GLIBCXX_USE_CXX11_ABI=0 -DEIGEN_MAX_ALIGN_BYTES=64 -Iutils -D GOOGLE_CUDA=1 -x cu -Xcompiler -fPIC -DNDEBUG --expt-relaxed-constexpr
/usr/local/lib/python3.7/dist-packages/tensorflow/include/absl/strings/string_view.h(337): warning: expression has no effect

/usr/local/lib/python3.7/dist-packages/tensorflow/include/absl/strings/string_view.h(347): warning: expression has no effect

/usr/local/lib/python3.7/dist-packages/tensorflow/include/absl/strings/string_view.h(529): warning: expression has no effect

/usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/platform/file_system.h(574): warning: overloaded virtual function "tensorflow::FileSystem::FilesExist" is only partially overridden in class "tensorflow::WrappedFileSystem"

/usr/local/lib/python3.7/dist-packages/tensorflow/inc

In [10]:
!make devox_test

python trilinear_devox/python/ops/trilinear_devox_ops_test.py
2021-11-15 03:35:27.202285: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 03:35:27.211007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 03:35:27.211589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] TrilinearDevoxTest.test_session
[  SKIPPED ] TrilinearDevoxTest.test_session
[ RUN      ] TrilinearDevoxTest.test_trilinear_devoxelize_forward
2021-11-15 03:35:27.215080: I tensorflow/core/platform/cpu_feature_gua

In [8]:
# from avg_vox.python.ops.avg_vox_ops import avg_voxelize_forward

trilinear_devox_ops = tf.load_op_library(trilinear_devox_ops_so_path)
# trilinear_devox_ops = tf.load_op_library('/root/gdrive/My Drive/_ECE 570/Project/ops/voxel-tf-ops/trilinear_devox/python/ops/_trilinear_devox_ops.so')
trilinear_devoxelize_forward = trilinear_devox_ops.trilinear_devox_forward
trilinear_devoxelize_backward = trilinear_devox_ops.trilinear_devox_backward

Note: We use R=5 rather than R=4 like in the avg_vox tests because of an
      issue in the backwards output (gradients of features/input). 

For the forward pass, the output is as expected (see below).
Note that we see 0 rather than 4 (i.e. expected_outs=[[[1,2,3,4]], ...], ...])
in `outs` because of the "clipping" when creating `cnt`. For more
information, see avg_vox/cc/avg_vox_kernels.cu.cc|GridStatsKernel 
or the avg_vox tests.

However, for the backwards pass, there was an inconsistency in the
1st channel of the 1st batch where all channels of each batch should
have the exact same R**3 values. For more detail, see the following
code block

In [9]:
###################
# Setup Constants #
###################
B = 2
C = 5
N = 4
# Note: We use R=5 rather than R=4 like in the avg_vox tests because of an
#       issue in the backwards output (gradients of features/input). 
#
#       For the forward pass, the output is as expected (see below).
#       Note that we see 0 rather than 4 (i.e. expected_outs=[[[1,2,3,4]]]])
#       in `outs` because of the "clipping" when creating `cnt`. For more
#       information, see avg_vox/cc/avg_vox_kernels.cu.cc|GridStatsKernel 
#       or the avg_vox tests.
#
#       However, for the backwards pass, there was an inconsistency in the
#       1st channel of the 1st batch where all channels of each batch should
#       have the exact same R**3 values. For more detail, see the following
#       code block.
R = 5 #4

# [B, C, R**3] = [2, 5, 64]
# features = tf.constant([[
#   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
#    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
#    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,]
#   ]], dtype=tf.float32)
# features = tf.repeat(tf.repeat(features, 5, axis=1), 2, axis=0)
# [B, C, R**3] = [2, 5, 125]
features = tf.constant([[
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 4]
 ]], dtype=tf.float32)
features = tf.repeat(tf.repeat(features, 5, axis=1), 2, axis=0)

# [B, 3, N] = [2, 3, 4]
coords = tf.constant([[[1, 2, 3, 4]]], dtype=tf.float32)
coords = tf.repeat(tf.repeat(coords, 3, axis=1), 2, axis=0)

# Attrs
resolution = tf.constant(R)
is_training = True

##################
# Call Operation #
##################
outs, inds, wgts = trilinear_devoxelize_forward(features, coords, resolution, is_training)

#################################################
# Get Gradients of Arbitrary Loss WRT Op Output #
#################################################
import time
time.sleep(1)
mse = tf.keras.losses.MeanSquaredError()
ones_label = tf.ones(shape=[B, C, N])
with tf.GradientTape() as tape:
  tape.watch(outs)
  loss = mse(ones_label, outs)
  dL_dout = tape.gradient(loss, outs)

####################################################
# Get Gradients of Loss WRT `features` input of Op #
####################################################
features_grad = trilinear_devoxelize_backward(dL_dout, inds, wgts, resolution)

#####################
# Print Out Tensors #
#####################
print("\n\n--------- Forward ---------\n")

print("outs\n----")
print(outs)
print()

print("\ninds\n----")
print(inds)
print()

print("wgts\n----")
print(wgts)
print()


# print("--------- Backward ---------\n")

print("dL/dout\n------")
print(dL_dout)
print()

print("features graient\n----------------")
print(features_grad)
print("one batch\n---------")
print(features_grad[0,:,:])
print("\n")




--------- Forward ---------

outs
----
tf.Tensor(
[[[1. 2. 3. 4.]
  [1. 2. 3. 4.]
  [1. 2. 3. 4.]
  [1. 2. 3. 4.]
  [1. 2. 3. 4.]]

 [[1. 2. 3. 4.]
  [1. 2. 3. 4.]
  [1. 2. 3. 4.]
  [1. 2. 3. 4.]
  [1. 2. 3. 4.]]], shape=(2, 5, 4), dtype=float32)


inds
----
tf.Tensor(
[[[ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]]

 [[ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]
  [ 31  62  93 124]]], shape=(2, 8, 4), dtype=int32)

wgts
----
tf.Tensor(
[[[1. 1. 1. 1.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[1. 1. 1. 1.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]], shape=(2, 8, 4), dtype=float32)

dL/dout
------
tf.Tensor(
[[[0.   0.05 0.1  0.15]
  [0.  

In [12]:
###################
# Setup Constants #
###################
B = 2
C = 5
N = 4
R = 4

# [B, C, R**3] = [2, 5, 64]
features = tf.constant([[
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,]
  ]], dtype=tf.float32)
features = tf.repeat(tf.repeat(features, 5, axis=1), 2, axis=0)

# [B, 3, N] = [2, 3, 4]
coords = tf.constant([[[1, 2, 3, 4]]], dtype=tf.float32)
coords = tf.repeat(tf.repeat(coords, 3, axis=1), 2, axis=0)

# Attrs
resolution = tf.constant(R)
is_training = True

##################
# Call Operation #
##################
outs, inds, wgts = trilinear_devoxelize_forward(features, coords, resolution, is_training)

#################################################
# Get Gradients of Arbitrary Loss WRT Op Output #
#################################################
import time
time.sleep(1)
mse = tf.keras.losses.MeanSquaredError()
ones_label = tf.ones(shape=[B, C, N])
with tf.GradientTape() as tape:
  tape.watch(outs)
  loss = mse(ones_label, outs)
  dL_dout = tape.gradient(loss, outs)

####################################################
# Get Gradients of Loss WRT `features` input of Op #
####################################################
features_grad = trilinear_devoxelize_backward(dL_dout, inds, wgts, resolution)

#####################
# Print Out Tensors #
#####################
print("\n\n--------- Forward ---------\n")

print("outs\n----")
print(outs)
print()

print("\ninds\n----")
print(inds)
print()

print("wgts\n----")
print(wgts)
print()


# print("--------- Backward ---------\n")

print("dL/dout\n------")
print(dL_dout)
print()

print("features graient\n----------------")
print(features_grad)
print("\n")




--------- Forward ---------

outs
----
tf.Tensor(
[[[1. 2. 3. 0.]
  [1. 2. 3. 0.]
  [1. 2. 3. 0.]
  [1. 2. 3. 0.]
  [1. 2. 3. 0.]]

 [[1. 2. 3. 0.]
  [1. 2. 3. 0.]
  [1. 2. 3. 0.]
  [1. 2. 3. 0.]
  [1. 2. 3. 0.]]], shape=(2, 5, 4), dtype=float32)


inds
----
tf.Tensor(
[[[21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]]

 [[21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]
  [21 42 63 84]]], shape=(2, 8, 4), dtype=int32)

wgts
----
tf.Tensor(
[[[1. 1. 1. 1.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[1. 1. 1. 1.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]], shape=(2, 8, 4), dtype=float32)

dL/dout
------
tf.Tensor(
[[[ 0.    0.05  0.1  -0.05]
  [ 0.    0.05  0.1  -0.05]
  [ 0.    0.05  0.1  -0.05]
  [ 0.    0